In [1]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, SEARCH_HASHTAGS_LIST, QUARTERS_DICT, POTATOES_CLEANED_DIR, FORBIDDEN_HASHTAGS_LIST, POTATOES_HASHTAGS_CLEAN, QUARTER_RESULTS, POTATOES_DIR, QUARTERS_ALL_POSTS, HASHTAGS_PER_USER_DIR, HASHTAG_FREQUENCY_DIR, CONNECTED_COMPONENTS_DIR, CONNECTED_COMPONENTS_PUBLIC_SPACES_DIR, HASHTAG_FREQUENCY_PUBLIC_SPACES_DIR, HASHTAGS_PER_USER_PUBLIC_SPACES_DIR
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv
import pandas as pd
import os

make_dirs()

def checkRow(rowHashtags, hashtags):
    if len(set.intersection(set(rowHashtags), set(hashtags))) > 0:
        return True
    else:
        return False

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Count posts per quarter before cleaning

In [3]:
post_count_dict = {}

dict = QUARTERS_DICT()
for quarter in dict:
    print('>> now processing', quarter)
    df = dict[quarter]
    post_count_dict[quarter] = len(df)
write_dict_to_csv(post_count_dict, "post_count", POTATOES_DIR, ['quarter', 'post_count'])

>> now loading: allach
>> now loading: altperlach
>> now loading: altstadt
>> now loading: amhart
>> now loading: arabellapark
>> now loading: au
>> now loading: aubing
>> now loading: bergamlaim
>> now loading: bogenhausen
>> now loading: borstei
>> now loading: daglfing
>> now loading: denning
>> now loading: fasanerie
>> now loading: fasangarten
>> now loading: feldmoching
>> now loading: forstenried
>> now loading: freiham
>> now loading: freimann
>> now loading: fröttmaning
>> now loading: fürstenried
>> now loading: giesing
>> now loading: hadern
>> now loading: haidhausen
>> now loading: harlaching
>> now loading: harras
>> now loading: harthof
>> now loading: hasenbergl
>> now loading: hellabrunn
>> now loading: herzogpark
>> now loading: isarvorstadt
>> now loading: johanneskirchen
>> now loading: kieferngarten
>> now loading: kirchtrudering
>> now loading: laim
>> now loading: lehel
>> now loading: lerchenau
>> now loading: lochhausen
>> now loading: ludwigsvorstadt
>> now lo

- merge different hashtags for same potato
- clean hashtags to clean
- remove forbidden hashtags

In [2]:
df_hashtags_clean = POTATOES_HASHTAGS_CLEAN()

selected_columns = ['owner_id','shortcode','hashtags', 'caption', 'post_url', 'is_video', 'likes']

df_post_counts = pd.DataFrame(columns=['quarter', 'post_count', 'post_count_cleaned'])

forbidden_hashtags = set(FORBIDDEN_HASHTAGS_LIST())
search_hashtags = SEARCH_HASHTAGS_LIST()


for index, row in df_hashtags_clean.iterrows():
    # merge files if different hashtags are used
    print('>> now processing', row.POTATOES)
    if len(row.HASHTAG)>0:
        df = pd.DataFrame()
        for hashtag in row.HASHTAG:
            df_new = QUARTER_RESULTS(hashtag)
            df_new_hashtags = df_new[selected_columns]
            df = df.append(df_new_hashtags,ignore_index=True)
    else:
        df = QUARTER_RESULTS(row.POTATOES)
        df =df[selected_columns]
    
    # clean irrelevant posts
    post_count = len(df)
    if (len(row.CLEAN)>0): # TODO: make separate row
        # df["to_clean"] = df.apply(lambda df_row: len(set.intersection(set(df_row.hashtags), set(row.CLEAN))), axis=1)
        # df.drop(df[df['to_clean'] != 0].index, inplace = True)

        mask = (df['hashtags'].apply(lambda x: checkRow(x, search_hashtags)))
        df = df[mask]
        
    post_count_cleaned = len(df)
    new_row = {'quarter': row.POTATOES, 'post_count': post_count, 'post_count_cleaned': post_count_cleaned}
    df_post_counts = df_post_counts.append(new_row, ignore_index=True)

    # clean forbidden hashtags
    df['hashtags'] = df['hashtags'].apply(lambda x: sorted(set(x).difference(forbidden_hashtags)))

    # write files
    write_df_to_csv(df, row.POTATOES, POTATOES_CLEANED_DIR)
    print(df_post_counts)
    
write_df_to_csv(df_post_counts, 'post_counts', POTATOES_DIR)

>> now processing allach
  quarter post_count post_count_cleaned
0  allach       6826               6826
>> now processing altperlach
      quarter post_count post_count_cleaned
0      allach       6826               6826
1  altperlach       1025               1025
>> now processing altstadt
      quarter post_count post_count_cleaned
0      allach       6826               6826
1  altperlach       1025               1025
2    altstadt       2664               2664
>> now processing amhart
      quarter post_count post_count_cleaned
0      allach       6826               6826
1  altperlach       1025               1025
2    altstadt       2664               2664
3      amhart       1039               1039
>> now processing arabellapark
        quarter post_count post_count_cleaned
0        allach       6826               6826
1    altperlach       1025               1025
2      altstadt       2664               2664
3        amhart       1039               1039
4  arabellapark       311

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


            quarter post_count post_count_cleaned
0            allach       6826               6826
1        altperlach       1025               1025
2          altstadt       2664               2664
3            amhart       1039               1039
4      arabellapark       3111               1856
5                au       4911               4911
6            aubing       9034               9034
7        bergamlaim       5846               5846
8       bogenhausen      29191              29191
9           borstei       1643               1643
10         daglfing       1482               1482
11          denning       3552                719
12        fasanerie       6531                928
13      fasangarten        868                596
14      feldmoching       4643               4643
15      forstenried       3063               3063
16          freiham       3390               3390
17         freimann       3007               3007
18      fröttmaning       2236               2236


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


            quarter post_count post_count_cleaned
0            allach       6826               6826
1        altperlach       1025               1025
2          altstadt       2664               2664
3            amhart       1039               1039
4      arabellapark       3111               1856
5                au       4911               4911
6            aubing       9034               9034
7        bergamlaim       5846               5846
8       bogenhausen      29191              29191
9           borstei       1643               1643
10         daglfing       1482               1482
11          denning       3552                719
12        fasanerie       6531                928
13      fasangarten        868                596
14      feldmoching       4643               4643
15      forstenried       3063               3063
16          freiham       3390               3390
17         freimann       3007               3007
18      fröttmaning       2236               2236


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


          quarter post_count post_count_cleaned
0          allach       6826               6826
1      altperlach       1025               1025
2        altstadt       2664               2664
3          amhart       1039               1039
4    arabellapark       3111               1856
..            ...        ...                ...
60      trudering      11509              11509
61   untermenzing       2561               2561
62  waldtrudering       2200               2200
63   werksviertel      12663              12663
64        westend     158196               1151

[65 rows x 3 columns]
>> now processing westkreuz
          quarter post_count post_count_cleaned
0          allach       6826               6826
1      altperlach       1025               1025
2        altstadt       2664               2664
3          amhart       1039               1039
4    arabellapark       3111               1856
..            ...        ...                ...
61   untermenzing       2561         

- Posts per user
- Frequency

In [ ]:
# see network